In [1]:
from global_funcs import load_program_config
from dask.distributed import Client
from dask_cuda import LocalCUDACluster




In [97]:
print("starting dask cluster...")
cluster = LocalCUDACluster()
client = Client(cluster)
print("finished starting dask cluster.")

import dask_cudf
import cudf as pd

print("loading configs...")
configs = load_program_config()

print("setting variables...")
clean_fasta_filepath = configs['clean_fasta_file']
output_dir = configs['output_dir']
project_name = configs['project_name']
base_col_names =  configs['base_col_names']
label_col_name = configs['label_col_name']
label_regex = configs['label_regex']
random_seed = configs['random_seed']

def extract_labels(df):
    df[label_col_name] = df[label_col_name].str.extract(label_regex).loc[:, 0]
    return df

print("reading data...")
df = dask_cudf.read_parquet(clean_fasta_filepath)
print("extracting all labels...")
df = df.map_partitions(extract_labels)


unq_labs
print("getting unique labels...")
unq_labs = df.sort_values(label_col_name)[label_col_name].unique().to_frame()
out_filepath = f"{output_dir}/{project_name}/data/unq_labels" 
print(f"saving unique labels to:\n{out_filepath}")
# _ = unq_labs.to_parquet(out_filepath)
out_filepath = f"{output_dir}/{project_name}/data/unq_labels.csv" 
print(f"saving unique labels to:\n{out_filepath}")
# _ = unq_labs.to_csv(out_filepath, index=False, single_file=True)

starting dask cluster...


/home/jcosme/miniconda3/envs/ds/lib/python3.9/site-packages/distributed/node.py:180: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40533 instead
  warnings.warn(
2022-05-17 15:52:11,691 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


finished starting dask cluster.
loading configs...
setting variables...
reading data...
extracting all labels...
getting unique labels...
saving unique labels to:
/media/jcosme/Data/mer-1/data/unq_labels
saving unique labels to:
/media/jcosme/Data/mer-1/data/unq_labels.csv


In [98]:
df = df.categorize(columns=[label_col_name])

In [101]:
df[label_col_name] = df[label_col_name].cat.codes

In [103]:
df[label_col_name].tail()

29995    9
29996    9
29997    9
29998    9
29999    9
Name: label, dtype: uint8

In [70]:
def encode_labels(df):
    from cuml.preprocessing import LabelEncoder
    le = LabelEncoder()
    le.fit(pd.read_parquet(f"{output_dir}/{project_name}/data/unq_labels"))
    df[label_col_name] = le.transform(df[label_col_name])
    return df
           

In [71]:
df.map_partitions(encode_labels)

ValueError: Metadata inference failed in `encode_labels`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
AttributeError('DataFrame object has no attribute dtype')

Traceback:
---------
  File "/home/jcosme/miniconda3/envs/ds/lib/python3.9/site-packages/dask/dataframe/utils.py", line 182, in raise_on_meta_error
    yield
  File "/home/jcosme/miniconda3/envs/ds/lib/python3.9/site-packages/dask/dataframe/core.py", line 6152, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
  File "/tmp/ipykernel_50784/2052587209.py", line 4, in encode_labels
    le.fit(pd.read_parquet(f"{output_dir}/{project_name}/data/unq_labels"))
  File "/home/jcosme/miniconda3/envs/ds/lib/python3.9/site-packages/cuml/internals/api_decorators

In [61]:
le = OneHotEncoder()

In [62]:
le.fit(unq_labs)

In [63]:
le.transform(df[[label_col_name]])

dask.array<concatenate, shape=(nan, nan), dtype=float32, chunksize=(nan, nan), chunktype=cupy.ndarray>

In [44]:
def encode_labels(df):
    le = LabelEncoder()
    le.fit(unq_labs[label_col_name].compute())
    df[label_col_name] = le.transform(df[label_col_name])
    return df

In [48]:
le.transform(df[label_col_name])

AttributeError: 'Series' object has no attribute '_column'

In [28]:
encoded.transform(df[[label_col_name]]).compute().to_dask_dataframe()

AttributeError: 'coo_matrix' object has no attribute 'to_dask_dataframe'

In [22]:
encoded.to_dask_dataframe().head()

ValueError: Shape along axis 1 must be known

In [11]:
df[label_col_name] = encoded

In [12]:
df.head()

2022-05-17 15:19:13,491 - distributed.worker - WARNING - Compute Failed
Key:       ('arg-reduce-1eca783f33f76ae120bc96de67d9471c', 0, 0)
Function:  arg_chunk
args:      (<cupyx.scipy.sparse.coo.coo_matrix object at 0x7efba6056670>, (1,), 0)
kwargs:    {}
Exception: "AxisError('axis 1 is out of bounds for array of dimension 0')"



AxisError: axis 1 is out of bounds for array of dimension 0

In [ ]:
encoded = encoded.argmax(axis=1)


In [ ]:
    out_filepath = f"{output_dir}/{project_name}/data/{project_name}"
    print(f"saving updated data (with labels extracted) to:\n{out_filepath}")
    _ = df.to_parquet(out_filepath)
    
    print("deleting dask df...")
    del df

    print("shutting down...")
    client.shutdown()
    client.close()
    print("label extraction complete!")